Install all necessary packages. 
Should only be done first time for notebook.

In [ ]:
# install.packages('tree', repos='http://cran.us.r-project.org')
# install.packages('ISLR', repos='http://cran.us.r-project.org')
# install.packages('MASS', repos='http://cran.us.r-project.org')
# install.packages('randomForest', repos='http://cran.us.r-project.org')
# install.packages('gbm', repos='http://cran.us.r-project.org')

**Fitting classification trees**

In [ ]:
library(tree)
library(ISLR)

We use the carseat dataset.
We create a binary variable high / low, if Sales <= 8 or > 8.

In [ ]:
attach(Carseats)
High=ifelse(Sales<=8,"No","Yes")
Carseats=data.frame(Carseats,High)

The Carseats data set tracks sales information for child car seats. 
It has 400 observations (each at a different store) and 11 variables:

* Sales: unit sales in thousands
* CompPrice: price charged by competitor at each location
* Income: community income level in 1000s of dollars
* Advertising: local ad budget at each location in 1000s of dollars
* Population: regional pop in thousands
* Price: price for car seats at each site
* ShelveLoc: Bad, Good or Medium indicates quality of shelving location
* Age: age level of the population
* Education: ed level at location
* Urban: Yes/No
* US: Yes/No

In [ ]:
summary(Carseats)

Fit a decision tree on the dataset.
We predict high / low sales, excluding of course the Sales variable.

In [ ]:
tree.carseats=tree(High~.-Sales,Carseats)

In [ ]:
summary(tree.carseats)

In [ ]:
plot(tree.carseats)
text(tree.carseats,pretty=0)

In [ ]:
tree.carseats

Create training / test set split, to estimate performance on independent test set

In [ ]:
set.seed(2)
train=sample(1:nrow(Carseats), 200)   # create training set with 200 samples (half of total)
Carseats.test=Carseats[-train,]       # create test set with other samples
High.test=High[-train]
tree.carseats=tree(High~.-Sales,Carseats,subset=train)
tree.pred=predict(tree.carseats,Carseats.test,type="class")
table(tree.pred,High.test)
(86+57)/200

Use cross-validation to estimate test error for different pruning strengths

The output for the CV is:
* size: number of terminal nodes |T|
* dev: CV error rate (confusing name!)
* k: alpha value  (-Inf must be error: 0 is lowest sensible value
* attr: attributes of class

In [ ]:
set.seed(3)
cv.carseats=cv.tree(tree.carseats,FUN=prune.misclass)   # we prune with missclassification error rate
names(cv.carseats)
cv.carseats

plot of missclassification error 

In [ ]:
par(mfrow=c(1,2))
plot(cv.carseats$size,cv.carseats$dev,type="b")
plot(cv.carseats$k,cv.carseats$dev,type="b")

Prune the tree to its optimal size, and plot the final result

In [ ]:
prune.carseats=prune.misclass(tree.carseats,best=9)
plot(prune.carseats)
text(prune.carseats,pretty=0)

Quality of pruned tree on test set (bettern than 0.72 of unpruned tree)

In [ ]:
tree.pred=predict(prune.carseats,Carseats.test,type="class")
table(tree.pred,High.test)
(94+60)/200

Indeed, a larger tree with 15 terminal nodes performs worse

In [ ]:
prune.carseats=prune.misclass(tree.carseats,best=15)
plot(prune.carseats)
text(prune.carseats,pretty=0)
tree.pred=predict(prune.carseats,Carseats.test,type="class")
table(tree.pred,High.test)
(86+62)/200

**Fitting regression trees**

We fit a regression tree to the Boston data set. It consists of
* CRIM - per capita crime rate by town
* ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS - proportion of non-retail business acres per town.
* CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX - nitric oxides concentration (parts per 10 million)
* RM - average number of rooms per dwelling
* AGE - proportion of owner-occupied units built prior to 1940
* DIS - weighted distances to five Boston employment centres
* RAD - index of accessibility to radial highways
* TAX - full-value property-tax rate per 10,000 dollar
* PTRATIO - pupil-teacher ratio by town
* B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
* LSTAT - % lower status of the population
* MEDV - Median value of owner-occupied homes in 1000's dollar

In [ ]:
library(MASS)
set.seed(1)
train = sample(1:nrow(Boston), nrow(Boston)/2)
tree.boston=tree(medv~.,Boston,subset=train)
summary(tree.boston)

In [ ]:
plot(tree.boston)
text(tree.boston,pretty=0)

The most complex tree is selected by cross-validation.

In [ ]:
cv.boston=cv.tree(tree.boston)
plot(cv.boston$size,cv.boston$dev,type='b')

Prune the tree anyway.

In [ ]:
prune.boston=prune.tree(tree.boston,best=5)
plot(prune.boston)
text(prune.boston,pretty=0)

In [ ]:
yhat=predict(tree.boston,newdata=Boston[-train,])
boston.test=Boston[-train,"medv"]
plot(yhat,boston.test)
abline(0,1)  # line with intercept 0 and slope 1
mean((yhat-boston.test)^2)

**Bagging and Random Forests**

Note that bagging is just a random forest with m=p.

In [ ]:
library(randomForest)
set.seed(1)
bag.boston=randomForest(medv~.,data=Boston,subset=train,mtry=13,importance=TRUE)
bag.boston

Already much better compared with a single pruned tree. (Half the MSE)

In [ ]:
yhat.bag = predict(bag.boston,newdata=Boston[-train,])
plot(yhat.bag, boston.test)
abline(0,1)
mean((yhat.bag-boston.test)^2)

Bagging with less trees reduces performance (MSE goes up).

In [ ]:
bag.boston=randomForest(medv~.,data=Boston,subset=train,mtry=13,ntree=25)
yhat.bag = predict(bag.boston,newdata=Boston[-train,])
mean((yhat.bag-boston.test)^2)

We now use a random forest, by changing mtry = 6 (instead of all predictors mtry = 13).
Performance further improves.

In [ ]:
set.seed(1)
rf.boston=randomForest(medv~.,data=Boston,subset=train,mtry=6,importance=TRUE)
yhat.rf = predict(rf.boston,newdata=Boston[-train,])
mean((yhat.rf-boston.test)^2)

In [ ]:
importance(rf.boston)

In [ ]:
varImpPlot(rf.boston)

**Boosting**

We now use boosting on the Boston data set.

In [ ]:
library(gbm)
set.seed(1)
boost.boston=gbm(medv~.,data=Boston[train,],distribution="gaussian",n.trees=5000,interaction.depth=4)

Relative influence is the average improvement (depends on the metric) of all splits containing that variable

In [ ]:
summary(boost.boston)

partial dependence plots: see https://christophm.github.io/interpretable-ml-book/pdp.html
integrate out the effect of other variables: f(X_s) = int f(X_s, X_c) dP(X_c) 

In [ ]:
par(mfrow=c(1,2))
plot(boost.boston,i="rm")
plot(boost.boston,i="lstat")

Better performance compared to boosting.
This dependence on randomness, so results can vary (could also lead to similar performance).
$\lambda = 0.001$ in this case.

In [ ]:
yhat.boost=predict(boost.boston,newdata=Boston[-train,],n.trees=5000)
mean((yhat.boost-boston.test)^2)

By taking $\lambda = 0.2$, test performance gets worse.

In [ ]:
boost.boston=gbm(medv~.,data=Boston[train,],distribution="gaussian",n.trees=5000,interaction.depth=4,shrinkage=0.2,verbose=F)
yhat.boost=predict(boost.boston,newdata=Boston[-train,],n.trees=5000)
mean((yhat.boost-boston.test)^2)

Eerste 2 oefeningen van boek kunnen ook gemaakt worden